In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

base_path = "/content/drive/MyDrive/Colab Notebooks/PFA"


Mounted at /content/drive


#Dataset personnalisé


In [3]:
class IncidentDataset(Dataset):
    def __init__(self, csv_path, transform=None):
        df = pd.read_csv(csv_path)
        self.paths = df["filepath"].tolist()
        self.labels = df["label"].tolist()
        self.transform = transform

        self.label2idx = {lbl: i for i, lbl in enumerate(sorted(set(self.labels)))}
        self.idx2label = {v: k for k, v in self.label2idx.items()}
        self.encoded_labels = [self.label2idx[lbl] for lbl in self.labels]

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.encoded_labels[idx]
        return img, label


#Transforms + DataLoaders


In [4]:
IMG_SIZE = 224

train_tfm = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_tfm = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_csv = f"{base_path}/train.csv"
val_csv   = f"{base_path}/val.csv"

train_ds = IncidentDataset(train_csv, transform=train_tfm)
val_ds   = IncidentDataset(val_csv,   transform=val_tfm)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print("✔️ DataLoaders prêts")
print("Nombres de classes :", len(train_ds.label2idx))


✔️ DataLoaders prêts
Nombres de classes : 4
